<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

##### Copyright 2024 Google LLC.

In [16]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Assisted programming with CodeGemma and KerasNLP

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/codegemma/code_assist_keras"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/codegemma/code_assist_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/codegemma/code_assist_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

CodeGemma is a variant of Gemma that is fine-tuned for coding tasks. This tutorial builds on the [Keras CodeGemma quickstart](https://colab.research.google.com/drive/11Va7W2Yl12JUnfx9YDJmy90kBu1i4rtw) and shows you more ways in which CodeGemma can assist your programming tasks.


## Setup

### Get access to CodeGemma

To complete this tutorial, you will first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on [kaggle.com](https://kaggle.com){:.external}.
* Select a Colab runtime with sufficient resources to run the Gemma 7B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Select the runtime

To run the CodeGemma 7B models, you'll need to have a paid Colab Pro plan which provides a runtime with an A100 GPU.

1. In the upper-right of the Colab window, select &#9662; (**Additional connection options**).
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **A100 GPU**.

### Configure your API key

To use Gemma, you must provide your Kaggle username and a Kaggle API key.

To generate a Kaggle API key, go to the **Account** tab of your Kaggle user profile and select **Create New Token**. This will trigger the download of a `kaggle.json` file containing your API credentials.

In Colab, select **Secrets** (🔑) in the left pane and add your Kaggle username and Kaggle API key. Store your username under the name `KAGGLE_USERNAME` and your API key under the name `KAGGLE_KEY`.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [17]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

SecretNotFoundError: Secret KAGGLE_USERNAME does not exist.

### Install dependencies

In [ ]:
!pip install -q -U keras-nlp

### Select a backend

Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. Using Keras 3, you can run workflows on one of three backends: TensorFlow, JAX, or PyTorch.

For this tutorial, configure the backend for JAX.

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

### Import packages

Import Keras and KerasNLP.

In [ ]:
import keras_nlp
import keras

# Run at half precision.
keras.config.set_floatx("bfloat16")

## CodeGemma 7B Model Examples

This section covers examples of using the pre-trained 7B CodeGemma model to help with coding tasks.


### Load the model

KerasNLP provides implementations of all three CodeGemma variants (2B and 7B pre-trained (PT) and 7B instruction-tuned (IT)) using [`GemmaCausalLM`](https://keras.io/api/keras_nlp/models/gemma/gemma_causal_lm/){:.external}, an end-to-end Gemma model for causal language modeling. A causal language model predicts the next token based on previous tokens.

For this example, load the `code_gemma_7b_en` model using the [`from_preset`](https://keras.io/api/keras_nlp/models/gemma/gemma_causal_lm/#frompreset-method){:.external} method.


In [ ]:
gemma_lm_7b = keras_nlp.models.GemmaCausalLM.from_preset("code_gemma_7b_en")

In [ ]:
gemma_lm_7b.summary()

The `from_preset` method instantiates the model from a preset architecture and weights.

### Code completion with Multi-line FIM

The PT CodeGemma models are trained on code infilling tasks. This section shows examples that use CodeGemma's multi-line fill-in the-middle (FIM) capability to autofill code at the specified cursor location based on the surrounding context.


As a first step, define constants and a prompt formatting helper function.

In [ ]:
# Formatting control tokens to specify cursor location
BEFORE_CURSOR = "<|fim_prefix|>"
AFTER_CURSOR = "<|fim_suffix|>"
AT_CURSOR = "<|fim_middle|>"
FILE_SEPARATOR = "<|file_separator|>"

# Define model stop tokens
END_TOKEN = gemma_lm_7b.preprocessor.tokenizer.end_token
stop_tokens = (BEFORE_CURSOR, AFTER_CURSOR, AT_CURSOR, FILE_SEPARATOR, END_TOKEN)
stop_token_ids = tuple(gemma_lm_7b.preprocessor.tokenizer.token_to_id(x) for x in stop_tokens)

def format_completion_prompt(before, after):
    return f"{BEFORE_CURSOR}{before}{AFTER_CURSOR}{after}{AT_CURSOR}"

#### Example 1 - Insert missing condition

The example code below to generate the Fibonacci sequence will not execute correctly if `n=1`:

```python
def fibonacci(n: int) -> int:
  if n == 0:
    return 0
  # The cursor is right before the e in the following line
  else:
    return fibonacci(n - 1) + fibonacci(n - 2)
```

Assuming that the cursor is at the beginning of line 4 (where the `else` clause is), then the content before and after the cursor is:

In [ ]:
before = """def fibonacci(n: int) -> int:\n  if n == 0:\n    return 0\n""" # Mind the spaces!
after = """\n  else:\n    return fibonacci(n - 1) + fibonacci(n-2)\n"""

In [ ]:
prompt = format_completion_prompt(before, after)
print(prompt)

Run the prompt.

In [ ]:
print(gemma_lm_7b.generate(prompt, stop_token_ids=stop_token_ids, max_length=128))

The model inserts the correct `elif` conidtion for `n=1` at the location of the cursor.

#### Example 2 - Complete DFS traversal algorithm

Auto-complete code for a depth-first search (DFS) tree traversal algorithm.

In [ ]:
before = """void dfs(node* root) {
  if (root->left) {
    dfs(root->left);
  }"""
after = """\nprintf("%d", root->value);
}"""

In [ ]:
prompt = format_completion_prompt(before, after)
print(prompt)

Run the prompt.

In [ ]:
print(gemma_lm_7b.generate(prompt, stop_token_ids=stop_token_ids, max_length=128))

### Code generation

In addition to code infilling, the CodeGemma 7B PT is model is also trained on natural language corpuses. You can use this to prompt the model to generate code.


In [ ]:
generation_prompt= """Write a rust function to identify non-prime numbers.
Examples:
>>> is_not_prime(2)
False
>>> is_not_prime(10)
True
pub fn is_not_prime(n: i32) -> bool {"""

In [ ]:
print(gemma_lm_7b.generate(generation_prompt, max_length=500))

## 7B IT model examples

This section uses the CodeGemma 7B Instruction-Tuned model for more advanced coding tasks. The CodeGemma 7B IT model is derived from the CodeGemma 7B PT model through supervised fine-tuning on code along with Reinforcement Learning with Human Feedback. This section covers examples of using this model for open-ended generation.

NOTE: If you are working through this tutorial in Colab and already have the 2B model loaded from above, restart your Colab Runtime by going to **Runtime** > **Disconnect and delete runtime** and re-connect to a new runtime. This frees up memory and prevents out-of-memory (OOM) issues. After connecting to a new runtime, re-run the Setup steps [from here](#set_environment_variables) before proceeding further.


### Load the IT model

Load the `code_gemma_instruct_7b_en` model using the `from_preset` method.

In [ ]:
gemma_lm_7b_it = keras_nlp.models.GemmaCausalLM.from_preset("code_gemma_instruct_7b_en")
gemma_lm_7b_it.summary()

IT models are trained with a specific formatter that annotates all instruction tuning examples with extra information to indicate roles and delineate turns in a conversation.

As a first step, define constants and a prompt formatting helper function.

In [ ]:
# Formatting control tokens for instruction tuning
START_OF_TURN_USER = "<start_of_turn>user"
END_OF_TURN = "<end_of_turn>"
START_OF_TURN_MODEL = "<start_of_turn>model"

# Formatting helper function
def format_instruction_prompt(context):
    return f"{START_OF_TURN_USER}\n{context}{END_OF_TURN}\n{START_OF_TURN_MODEL}\n"

### Code translation


In [ ]:
context1 = """
You are an experienced C and Python programmer. Convert the following Python code into C.
```python
def factorial(n):
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result
```\n"""

Format the prompt.

In [ ]:
prompt1 = format_instruction_prompt(context1)
print(prompt1)

Run the prompt.

In [ ]:
print(gemma_lm_7b_it.generate(prompt1, max_length=500))

### Code vulnerability detection


In [ ]:
context2 = """
You are an experienced C++ programmer hunting for vulnerable code. Is the following code vulnerable? Explain your reasoning.
```cpp
int i;
unsigned int numWidgets;
Widget **WidgetList;

numWidgets = GetUntrustedSizeValue();
if ((numWidgets == 0) || (numWidgets > MAX_NUM_WIDGETS)) {
    ExitError("Incorrect number of widgets requested!");
}
WidgetList = (Widget **) malloc(numWidgets * sizeof(Widget *));
printf("WidgetList ptr=%p\n", WidgetList);
for (i = 0; i < numWidgets; i++) {
    WidgetList[i] = InitializeWidget();
}
WidgetList[numWidgets] = NULL;
showWidgets(WidgetList);
```\n"""

Format the prompt.

In [ ]:
prompt2 = format_instruction_prompt(context2)
print(prompt2)

In [ ]:
print(gemma_lm_7b_it.generate(prompt2, max_length=1000))

The model detects a potential vulnerability in the code and provides code changes to mitigate it.

## Summary

This tutorial walked you through using CodeGemma for a variety of coding tasks. To learn more about CodeGemma:

* Refer to the [CodeGemma model card](https://ai.google.dev/gemma/docs/codegemma/model_card) for the technical specs of the CodeGemma models.
* Learn more about how to use CodeGemma in VertexAI [here](https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_codegemma_deployment_on_vertex.ipynb).
* Check out the [Keras CodeGemma quickstart](https://ai.google.dev/gemma/docs/codegemma/keras_quickstart).